## Pytorch GoogleNet
- Inception Module
    - 기초 모듈 : 1x1 conv / 3x3 conv / 5x5 conv / 3x3 maxpool
    - 차원 감소 더한 모듈
        - 1x1 conv / 1x1conv + 3x3conv / 1x1conv + 5x5conv / 3x3 maxpool + 1x1conv

## Inception Module
- conv_1, conv_1_3, conv_1_5, max_3_1 (4가지 가지치기)

In [2]:
import torch
import torch.optim as optim
import torch.nn as nn

batch_size = 128

class Inception_module(nn.Module):
    def __init__(self, in_dim, out_dim_1, mid_dim_3, out_dim_3, mid_dim_5, out_dim_5, pool):
        super(Inception_module, self).__init__()

        self.m_conv_1 = self.conv_1(in_dim, out_dim_1)
        self.m_conv_1_3 = self.conv_1_3(in_dim, mid_dim_3, out_dim_3)
        self.m_conv_1_5 = self.conv_1_5(in_dim, mid_dim_5, out_dim_5)
        self.m_max_3_1 = self.max_3_1(in_dim, pool)

    def forward(self, x):
        out_1 = self.m_conv_1(x)
        out_2 = self.m_conv_1_3(x)
        out_3 = self.m_conv_1_5(x)
        out_4 = self.m_max_3_1(x)
        output = torch.cat([out_1, out_2, out_3, out_4], 1)
        return output

    # 1x1 conv
    def conv_1(self, i_dim, o_dim):
        model = nn.Sequential(
            nn.Conv2d(i_dim, o_dim, kernel_size = 1, padding = 1),
            nn.ReLU()
        )
        return model

    # 1x1 conv + 3x3 conv
    def conv_1_3(self, i_dim, m_dim, o_dim):
        model = nn.Sequential(
            nn.Conv2d(i_dim, m_dim, kernel_size = 1, padding = 1),
            nn.ReLU(),
            nn.Conv2d(m_dim, o_dim, kernel_size = 3, padding = 1, stride = 1),
            nn.ReLU()
        )
        return model

    # 1x1 conv + 5x5 conv
    def conv_1_5(self, i_dim, m_dim, o_dim):
        model = nn.Sequential(
            nn.Conv2d(i_dim, m_dim, kernel_size = 1, padding = 1),
            nn.ReLU(),
            nn.Conv2d(m_dim, o_dim, kernel_size = 5, padding = 1, stride = 2),
            nn.ReLU()
        )
        return model

    # 3x3 maxpool + 1x1 conv
    def max_3_1(self, i_dim, o_dim):
        model = nn.Sequential(
            nn.MaxPool2d(3,1,1),
            nn.Conv2d(i_dim, o_dim, 1, 1),
            nn.ReLU()
        )
        return model

## GoogleNet
- Inception Module 
    - layer 앞부분 (CMCM)
    - Inception Module 2개 + Maxpool
    - Inception Module 5개 + Maxpool
    - Inception Module 2개 + Avgpool
    - FC Layer

In [4]:
class GoogleNet(nn.Module):
    def __init__(self, base_dim, num_classes=2):
        super(GoogleNet, self).__init__()
        
        # layer 종류 1 - 앞부분
        self.layer_1 = nn.Sequential(
            nn.Conv2d(3, base_dim, 7, 2, 3),
            nn.MaxPool2d(3, 2, 1),
            nn.Conv2d(base_dim, base_dim * 3, 3, 1, 1),
            nn.MaxPool2d(3, 2, 1),
        )

        # Inception Module 2개 + Maxpool
        self.layer_2 = nn.Sequential(
            Inception_module(base_dim * 3, 64, 96, 128, 16, 32, 32), # in_dim, out_dim_1, mid_dim_3, out_dim_3, mid_dim_5, out_dim_5, pool
            Inception_module(base_dim * 4, 128, 128, 192, 32, 96, 64), # in_dim, out_dim_1, mid_dim_3, out_dim_3, mid_dim_5, out_dim_5, pool
            nn.MaxPool2d(3, 2, 1),
        )

        # Inception Module 5개 + Maxpool
        self.layer_3 = nn.Sequential(
            Inception_module(480, 192, 96, 208, 16, 48, 64),
            Inception_module(512, 160, 112, 224, 24, 64, 64),
            Inception_module(512, 128, 128, 256, 24, 64, 64),
            Inception_module(512, 112, 144, 288, 32, 64, 64),
            Inception_module(528, 256, 160, 320, 32, 128, 128),
            nn.MaxPool2d(3, 2, 1),
        )

        # Inception Module 2개 + Avgpool
        self.layer_4 = nn.Sequential(
            Inception_module(832, 256, 160, 320, 32, 128, 128),
            Inception_module(832, 384, 192, 384, 48, 128, 128),
            nn.AvgPool2d(7, 1),

        )

        # FC Layer
        self.layer_5 = nn.Dropout2d(0, 4)
        self.fc_layer = nn.Linear(1024, 1000)

    def forward(self, x):
        out = self.layer_1(x)
        out = self.layer_2(out)
        out = self.layer_3(out)
        out = self.layer_4(out)
        out = self.layer_5(out)
        out = out.view(batch_size, -1)
        out = self.fc_layer(out)

        return out

In [11]:
from torchsummary import summary as summary_

if __name__ == "__main__":
    model = GoogleNet(16)
    print(model)

GoogleNet(
  (layer_1): Sequential(
    (0): Conv2d(3, 16, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
    (1): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (2): Conv2d(16, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (layer_2): Sequential(
    (0): Inception_module(
      (m_conv_1): Sequential(
        (0): Conv2d(48, 64, kernel_size=(1, 1), stride=(1, 1), padding=(1, 1))
        (1): ReLU()
      )
      (m_conv_1_3): Sequential(
        (0): Conv2d(48, 96, kernel_size=(1, 1), stride=(1, 1), padding=(1, 1))
        (1): ReLU()
        (2): Conv2d(96, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (3): ReLU()
      )
      (m_conv_1_5): Sequential(
        (0): Conv2d(48, 16, kernel_size=(1, 1), stride=(1, 1), padding=(1, 1))
        (1): ReLU()
        (2): Conv2d(16, 32, kernel_size=(5, 5), stride=(2, 2), padding=(1, 1))
     